<a href="https://colab.research.google.com/github/TarteTatin1212/merge_face_and_metadata/blob/main/merge_face_and_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from os.path import isfile, join
from google.colab import drive


In [ ]:
# Googleドライブをマウント
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# YouTubeのメタデータが保存されているCSVファイルのパス
csv_filepath = '/content/drive/MyDrive/Horita_Semi/Senior_Project/saved_data/csv/japanese_20231004_224243.csv'

# 顔認識されたサムネイルのディレクトリパス
recognized_faces_dir = '/content/drive/MyDrive/Horita_Semi/Senior_Project/FaceRecognitionResults/RecognizedFaces'

# 顔認識されなかったサムネイルのディレクトリパス
unrecognized_faces_dir = '/content/drive/MyDrive/Horita_Semi/Senior_Project/FaceRecognitionResults/UnrecognizedFaces'


In [ ]:
# 顔認識されたサムネイルのファイル名（動画ID）をリストに保存
recognized_faces = [f.replace('.jpg', '') for f in os.listdir(recognized_faces_dir) if isfile(join(recognized_faces_dir, f)) and f.endswith('.jpg')]

# 顔認識されなかったサムネイルのファイル名（動画ID）をリストに保存
unrecognized_faces = [f.replace('.jpg', '') for f in os.listdir(unrecognized_faces_dir) if isfile(join(unrecognized_faces_dir, f)) and f.endswith('.jpg')]


In [ ]:
# それぞれのリストからデータフレームを作成
recognized_faces_df = pd.DataFrame(recognized_faces, columns=['video_id'])
recognized_faces_df['face_detected'] = True

unrecognized_faces_df = pd.DataFrame(unrecognized_faces, columns=['video_id'])
unrecognized_faces_df['face_detected'] = False

# 二つのデータフレームを結合
faces_df = pd.concat([recognized_faces_df, unrecognized_faces_df], ignore_index=True)

# 結果の表示（必要に応じて）
print(faces_df.head())


      video_id  face_detected
0  CqIsq7QFCfY           True
1  YpE7g50kCdQ          False
2  kAZHeXodJj0          False
3  DCuoWYeneuc          False
4  SkyrTccqmXU          False


In [ ]:
# YouTubeのメタデータを読み込む
youtube_metadata = pd.read_csv(csv_filepath)

# 結果の表示（必要に応じて）
print(youtube_metadata.head())


      video_id                                              title  \
0  DCuoWYeneuc                                      地上波で冠番組はじめました   
1  SkyrTccqmXU      『THE FIRST TAKE』ユートピア公開されました😈🙌🏻 #imase #ユートピア   
2  CqIsq7QFCfY                            【最終戦セレモニー】原辰徳監督から挨拶【巨人】   
3  QIkN5LdKhNY                           【最終戦セレモニー】原辰徳監督へ花束贈呈【巨人】   
4  YpE7g50kCdQ  【速報】甲府がACL初勝利！ 90分に待っていたドラマ、長谷川元希のゴールが新たな歴史を切り...   

                     channelTitle  \
0  ATARASHII GAKKO! - 新しい学校のリーダーズ   
1                           imase   
2          DRAMATIC BASEBALL 2023   
3          DRAMATIC BASEBALL 2023   
4                      DAZN Japan   

                                     thumbnailUrl  viewCount  
0  https://i.ytimg.com/vi/DCuoWYeneuc/default.jpg      35192  
1  https://i.ytimg.com/vi/SkyrTccqmXU/default.jpg      11839  
2  https://i.ytimg.com/vi/CqIsq7QFCfY/default.jpg     347897  
3  https://i.ytimg.com/vi/QIkN5LdKhNY/default.jpg     143720  
4  https://i.ytimg.com/vi/YpE7g50kCdQ/default.j

In [ ]:
# youtube_metadataのカラム名を表示
print(youtube_metadata.columns.tolist())


['video_id', 'title', 'channelTitle', 'thumbnailUrl', 'viewCount']


In [ ]:
# 動画IDをキーにしてYouTubeのメタデータと顔認識の結果をマージ（カラム名が異なる場合）
merged_data = pd.merge(youtube_metadata, faces_df, left_on='video_id', right_on='video_id', how='left')

# face_detectedカラムのNaNをFalseに置換（顔が検出されなかった動画）
merged_data['face_detected'].fillna(False, inplace=True)

# 結果の表示
print(merged_data.head())


      video_id                                              title  \
0  DCuoWYeneuc                                      地上波で冠番組はじめました   
1  SkyrTccqmXU      『THE FIRST TAKE』ユートピア公開されました😈🙌🏻 #imase #ユートピア   
2  CqIsq7QFCfY                            【最終戦セレモニー】原辰徳監督から挨拶【巨人】   
3  QIkN5LdKhNY                           【最終戦セレモニー】原辰徳監督へ花束贈呈【巨人】   
4  YpE7g50kCdQ  【速報】甲府がACL初勝利！ 90分に待っていたドラマ、長谷川元希のゴールが新たな歴史を切り...   

                     channelTitle  \
0  ATARASHII GAKKO! - 新しい学校のリーダーズ   
1                           imase   
2          DRAMATIC BASEBALL 2023   
3          DRAMATIC BASEBALL 2023   
4                      DAZN Japan   

                                     thumbnailUrl  viewCount  face_detected  
0  https://i.ytimg.com/vi/DCuoWYeneuc/default.jpg      35192          False  
1  https://i.ytimg.com/vi/SkyrTccqmXU/default.jpg      11839          False  
2  https://i.ytimg.com/vi/CqIsq7QFCfY/default.jpg     347897           True  
3  https://i.ytimg.com/vi/QIkN5LdKhNY/default.jpg 

In [ ]:
# マージされたデータをCSVとして保存
merged_data.to_csv('/content/drive/MyDrive/Horita_Semi/Senior_Project/merged_data.csv', index=False)

# 保存したファイルのパスを表示
print("Data saved to /content/drive/MyDrive/Horita_Semi/Senior_Project/merged_data.csv")


Data saved to /content/drive/MyDrive/Horita_Semi/Senior_Project/merged_data.csv
